In [2]:
import requests
import pymysql

# 1. API 호출
url = "https://chargeinfo.ksga.org/ws/statistics/ev/list"
data = requests.get(url).json()
results = data['result']

# 2. 지역 코드 → 지역 이름 매핑
region_map = {
    "rgn11000": "서울", "rgn26000": "부산", "rgn27000": "대구", "rgn28000": "인천",
    "rgn29000": "광주", "rgn30000": "대전", "rgn31000": "울산", "rgn36000": "세종",
    "rgn41000": "경기", "rgn42000": "강원", "rgn43000": "충북", "rgn44000": "충남",
    "rgn45000": "전북", "rgn46000": "전남", "rgn47000": "경북", "rgn48000": "경남",
    "rgn50000": "제주"
}

# 3. 연도별 데이터 저장
yearly_data = {}

for item in results:
    yyyyMm = item["yyyyMm"]
    year = yyyyMm[:4]  # 연도만 추출

    # 조건: 각 연도의 12월 데이터 또는 202407 (현재 최신)
    if yyyyMm == '202407' or yyyyMm.endswith('12'):
        if year not in yearly_data:
            yearly_data[year] = {region: 0 for region in region_map.values()}

        for code, region in region_map.items():
            yearly_data[year][region] = item.get(code, 0)

# 4. DB 연결
conn = pymysql.connect(
    host='127.0.0.1',
    user='play',
    passwd='123',
    database='sk15',
    port=3306
)
cursor = conn.cursor()

# 5. INSERT (연도만 저장)
for year in sorted(yearly_data.keys()):
    for region in region_map.values():
        count = yearly_data[year][region]
        sql = """
            INSERT INTO ev_car_statistics_yearly (year, region, count)
            VALUES (%s, %s, %s)
        """
        cursor.execute(sql, (year, region, count))

# 6. 마무리
conn.commit()
cursor.close()
conn.close()
